In [2]:
import xarray as xr
import numpy as np
import rasterio as rio
import rioxarray
from affine import Affine
import geopandas as gpd
import pandas as pd
import xagg as xa
from rasterstats import zonal_stats
import matplotlib.pyplot as plt

In [3]:
dpath = '/home/nick/workspace/climate_office/mt-drought-triggers/data/'
ds = xr.open_dataset(dpath + "All_Droughts_MT.nc")
counties = gpd.read_file("zip://" + dpath + "MT_Counties.zip")
transform = rio.open(dpath + "All_Droughts_MT.nc").transform

In [4]:
counties = counties[['NAME', 'COUNTYNUMB', 'geometry']]
counties.index = counties['COUNTYNUMB']
counties.drop(columns=['COUNTYNUMB'], inplace=True)

In [44]:
ds_slice = ds.sel(time=slice('2001-01-01', '2020-12-31'))
ds_djf = ds_slice['DM'].loc[ds_slice["time.month"].isin([12, 1, 2])]
ds_mam = ds_slice['DM'].loc[ds_slice["time.month"].isin([3, 4, 5])]
ds_jja = ds_slice['DM'].loc[ds_slice["time.month"].isin([6, 7, 8])]
ds_son = ds_slice['DM'].loc[ds_slice["time.month"].isin([9, 10, 11])]

In [48]:
drought = ds_slice['DM'].where(ds_slice['DM'] >= 2).sum(dim='time', skipna=True, min_count=1)
drought_djf = ds_djf.where(ds_djf >= 2).sum(dim='time', skipna=True, min_count=1)
drought_mam = ds_mam.where(ds_mam >= 2).sum(dim='time', skipna=True, min_count=1)
drought_jja = ds_jja.where(ds_jja >= 2).sum(dim='time', skipna=True, min_count=1)
drought_son = ds_son.where(ds_son >= 2).sum(dim='time', skipna=True, min_count=1)

In [50]:
drought.rio.to_raster('data/D2_Drought_Freq_2001-2020.tif')
drought_djf.rio.to_raster('data/D2_Drought_Freq_DJF_2001-2020.tif')
drought_mam.rio.to_raster('data/D2_Drought_Freq_MAM_2001-2020.tif')
drought_jja.rio.to_raster('data/D2_Drought_Freq_JJA_2001-2020.tif')
drought_son.rio.to_raster('data/D2_Drought_Freq_SON_2001-2020.tif')